In [37]:
import os
import pandas as pd
from pandas import Series, DataFrame
import country_converter as coco
import warnings
warnings.filterwarnings("ignore")

In [38]:
## Get current working directory
print("Current working directory:", os.getcwd())

## Find path to raw data (for the raw files)
raw_data_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'raw_data'))

## Find path to folder for saving cleaned csv
cleaned_data_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'cleaned_data'))

Current working directory: /Users/jennagreene/Documents/GitHub/HATCH_data/reading_files


In [39]:
#Ignore footer of 34 because from 2017 - 2050 are projections; 2016 is last year of empirical data
iea_path = os.path.join(raw_data_path, 'global-air-conditioner-stock-1990-2050.csv')
iea = pd.read_csv(iea_path, header = 3, skipfooter= 34)

In [40]:
iea = iea.set_index('Year').transpose()
iea = iea.reset_index(drop = False)

In [41]:
iea = iea.rename(columns = {'index': 'Country Name'})

iea

Year,Country Name,1990,1991,1992,1993,1994,1995,1996,1997,1998,...,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
0,United States,227.02,224.22,230.77,237.33,244.60,251.28,258.77,266.75,276.48,...,340.37,345.62,345.92,349.03,353.28,357.83,355.65,359.64,367.76,373.59
1,China,78.55,80.99,86.98,90.20,97.76,102.23,111.16,117.25,124.17,...,252.47,274.79,308.58,342.92,378.51,419.33,457.03,492.13,531.23,569.38
2,Japan and Korea,106.53,114.06,118.35,122.80,126.95,131.95,135.70,140.08,142.34,...,179.06,182.63,183.94,189.87,194.68,198.26,202.11,203.19,204.91,206.96
3,European Union,43.77,44.33,45.92,46.84,47.93,49.28,51.08,53.57,56.36,...,83.43,86.19,88.11,88.22,88.46,88.11,88.88,90.48,93.30,96.52
4,India,2.47,2.76,3.14,3.59,4.09,4.72,5.41,6.20,7.07,...,12.17,13.08,14.25,15.48,16.67,18.05,19.50,21.52,23.74,26.56
5,Indonesia,1.86,1.97,2.08,2.27,2.44,2.68,2.95,3.13,2.98,...,4.82,5.20,5.60,6.06,6.67,7.67,8.74,9.80,10.91,12.14
6,Mexico,4.60,5.19,5.47,5.26,5.32,5.11,5.11,5.44,6.22,...,10.52,11.11,11.12,11.31,11.78,12.56,13.63,14.30,15.12,15.84
7,Brazil,7.24,7.73,7.83,8.46,9.40,10.53,11.04,11.79,12.13,...,18.16,19.97,21.94,25.11,27.25,29.34,31.12,31.34,29.79,27.40
8,Middle East,12.10,13.38,14.49,15.11,15.24,15.64,16.14,16.91,19.01,...,29.97,33.54,40.03,42.09,41.05,42.41,43.60,44.62,45.54,47.03
9,Rest of world,91.55,94.24,96.36,99.90,102.03,104.28,107.13,111.13,113.53,...,174.33,183.06,189.35,198.96,207.71,218.41,228.40,235.30,241.15,246.30


In [42]:
#iea.rename(columns = {'Year': 'Country Name'}, inplace = True)

In [43]:

iea['Data Source'] = 'IEA'
iea['Technology Name'] = 'Home Air Conditioning'
iea['Spatial Scale'] = 'National'
iea['Metric'] = 'Number of Units'
iea['Unit'] = 'million'
iea['Country Code'] = coco.convert(names=iea['Country Name'], to='iso2')
iea = iea.dropna(subset=['Country Code'])


European Union not found in regex
Middle East not found in regex
Rest of world not found in regex


In [44]:
iea_filtered = iea[iea['Country Code'] != 'not found']
iea_filtered = iea_filtered[iea_filtered['Country Code'] != 'JP']

iea = iea.merge(iea_filtered[['Country Code']], on='Country Code', how='inner')
iea

Year,Country Name,1990,1991,1992,1993,1994,1995,1996,1997,1998,...,2013,2014,2015,2016,Data Source,Technology Name,Spatial Scale,Metric,Unit,Country Code
0,United States,227.02,224.22,230.77,237.33,244.60,251.28,258.77,266.75,276.48,...,355.65,359.64,367.76,373.59,IEA,Home Air Conditioning,National,Number of Units,million,US
1,China,78.55,80.99,86.98,90.20,97.76,102.23,111.16,117.25,124.17,...,457.03,492.13,531.23,569.38,IEA,Home Air Conditioning,National,Number of Units,million,CN
2,India,2.47,2.76,3.14,3.59,4.09,4.72,5.41,6.20,7.07,...,19.50,21.52,23.74,26.56,IEA,Home Air Conditioning,National,Number of Units,million,IN
3,Indonesia,1.86,1.97,2.08,2.27,2.44,2.68,2.95,3.13,2.98,...,8.74,9.80,10.91,12.14,IEA,Home Air Conditioning,National,Number of Units,million,ID
4,Mexico,4.60,5.19,5.47,5.26,5.32,5.11,5.11,5.44,6.22,...,13.63,14.30,15.12,15.84,IEA,Home Air Conditioning,National,Number of Units,million,MX
5,Brazil,7.24,7.73,7.83,8.46,9.40,10.53,11.04,11.79,12.13,...,31.12,31.34,29.79,27.40,IEA,Home Air Conditioning,National,Number of Units,million,BR


In [45]:
iea.columns.name = None

iea['ID'] = iea['Technology Name'] + '_' + iea['Metric'] + '_' + iea['Country Code']

iea.set_index('ID', inplace = True, drop = False)



In [46]:
iea

,Country Name,1990,1991,1992,1993,1994,1995,1996,1997,1998,...,2014,2015,2016,Data Source,Technology Name,Spatial Scale,Metric,Unit,Country Code,ID
ID,,,,,,,,,,,,,,,,,,,,,
Home Air Conditioning_Number of Units_US,United States,227.02,224.22,230.77,237.33,244.60,251.28,258.77,266.75,276.48,...,359.64,367.76,373.59,IEA,Home Air Conditioning,National,Number of Units,million,US,Home Air Conditioning_Number of Units_US
Home Air Conditioning_Number of Units_CN,China,78.55,80.99,86.98,90.20,97.76,102.23,111.16,117.25,124.17,...,492.13,531.23,569.38,IEA,Home Air Conditioning,National,Number of Units,million,CN,Home Air Conditioning_Number of Units_CN
Home Air Conditioning_Number of Units_IN,India,2.47,2.76,3.14,3.59,4.09,4.72,5.41,6.20,7.07,...,21.52,23.74,26.56,IEA,Home Air Conditioning,National,Number of Units,million,IN,Home Air Conditioning_Number of Units_IN
Home Air Conditioning_Number of Units_ID,Indonesia,1.86,1.97,2.08,2.27,2.44,2.68,2.95,3.13,2.98,...,9.80,10.91,12.14,IEA,Home Air Conditioning,National,Number of Units,million,ID,Home Air Conditioning_Number of Units_ID
Home Air Conditioning_Number of Units_MX,Mexico,4.60,5.19,5.47,5.26,5.32,5.11,5.11,5.44,6.22,...,14.30,15.12,15.84,IEA,Home Air Conditioning,National,Number of Units,million,MX,Home Air Conditioning_Number of Units_MX
Home Air Conditioning_Number of Units_BR,Brazil,7.24,7.73,7.83,8.46,9.40,10.53,11.04,11.79,12.13,...,31.34,29.79,27.40,IEA,Home Air Conditioning,National,Number of Units,million,BR,Home Air Conditioning_Number of Units_BR


In [47]:
output_file = 'airconditioning.csv'
output_file_path = os.path.join(cleaned_data_path, output_file)

iea.to_csv(output_file_path, index = False)
print("Data saved to:", output_file_path)

Data saved to: /Users/jennagreene/Documents/GitHub/HATCH_data/cleaned_data/airconditioning.csv
